In [1]:
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import scanpy as sc
import scvi
import torch

from pytorch_lightning.loggers import WandbLogger
import wandb
from dotenv import load_dotenv

import session_info
import warnings
from pyprojroot.here import here

warnings.filterwarnings("ignore")
sys.path.insert(1, str(here('bin')))

torch.set_float32_matmul_precision('high')

#plt.style.use(['science','nature','no-latex'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

load_dotenv(here('.env'))

True

In [2]:
class CustomWandbLogger(WandbLogger):
    @property
    def save_dir(self):
        return self.experiment.dir

In [3]:
scvi.settings.seed = 0
print("Last run with scvi-tools version:", scvi.__version__)

Seed set to 0


Last run with scvi-tools version: 1.1.2


**Setting parameters** 

In [4]:
overwriteFigures = True
overwriteData = True

In [5]:
cellGroup = 'Plasma'
workDir = os.getcwd()

In [6]:
annotationLevel = os.path.basename(os.path.normpath(workDir))

In [7]:
workDir

'/home/jupyter/Inflammation-PBMCs-Atlas/02_cell_annotation/02_fromCellLineages_to_CellTypes/Step1'

**Load data**

In [8]:
adata = sc.read_h5ad(here(f"{workDir}/{cellGroup}/results/01_{cellGroup}_normalized_HVGsubset.h5ad"))
adata

AnnData object with n_obs × n_vars = 21848 × 2404
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet', 'S_score', 'G2M_score', 'phase'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable'
    uns: 'chemistr

## scVI integration

**scVI parameters**

In [9]:
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.Trainer.html#scvi.train.Trainer
setup_kwargs = dict(
    layer="counts", 
    batch_key='libraryID', 
    categorical_covariate_keys = ['studyID', 'chemistry', 'disease', 'sampleID'],
    labels_key = 'chemistry'    
)

scvi_kwargs = dict(n_hidden=512,
                   n_latent=20, # reduceds after step 00 (main)
                   n_layers=2,
                   gene_likelihood='nb',
                   dispersion='gene-label')

trainer_kwargs = dict(
    checkpointing_monitor = 'elbo_validation',
    early_stopping_monitor = 'reconstruction_loss_validation',
    early_stopping_patience = 10,
    early_stopping_min_delta=0.1,
    early_stopping = True,
    max_epochs = 1000,

    #logger = # wandb
)
# https://docs.scvi-tools.org/en/stable/api/reference/scvi.train.TrainingPlan.html#scvi.train.TrainingPlan
plan_kwargs = dict(
    lr = 5e-4,
    #reduce_lr_on_plateau = True
)

# https://docs.scvi-tools.org/en/stable/api/reference/scvi.module.VAE.html#scvi.module.VAE
#vae = dict(
#    use_layer_norm='both',
#    use_batch_norm='none',
#    encode_covariates=True,
#    deeply_inject_covariates=False
#)
parameter_dict = setup_kwargs | scvi_kwargs | trainer_kwargs | plan_kwargs

**wandb parameter**

In [10]:
run_name = f"{annotationLevel}_{cellGroup}"
run_name

'Step1_Plasma'

In [11]:
logger = CustomWandbLogger(name = run_name, project='inflammation_atlas_R1', config=parameter_dict)

**Running integration**

In [12]:
scvi.model.SCVI.setup_anndata(adata, 
                              **setup_kwargs)

In [13]:
model = scvi.model.SCVI(adata, **scvi_kwargs)

In [14]:
model.train(logger=logger, plan_kwargs = plan_kwargs, **trainer_kwargs)

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores


IPU available: False, using: 0 IPUs


HPU available: False, using: 0 HPUs


wandb: Currently logged in as: dav1989 (inflammation). Use `wandb login --relogin` to force relogin


...

Epoch 48/1000:   5%|▍         | 47/1000 [02:03<38:38,  2.43s/it, v_num=mr0f, train_loss_step=1.33e+3, train_loss_epoch=1.3e+3]

Epoch 48/1000:   5%|▍         | 48/1000 [02:06<37:35,  2.37s/it, v_num=mr0f, train_loss_step=1.33e+3, train_loss_epoch=1.3e+3]

Epoch 48/1000:   5%|▍         | 48/1000 [02:06<37:35,  2.37s/it, v_num=mr0f, train_loss_step=1.28e+3, train_loss_epoch=1.3e+3]

Epoch 48/1000:   5%|▍         | 48/1000 [02:06<41:40,  2.63s/it, v_num=mr0f, train_loss_step=1.28e+3, train_loss_epoch=1.3e+3]


Monitored metric reconstruction_loss_validation did not improve in the last 10 records. Best score: 1370.138. Signaling Trainer to stop.


In [15]:
wandb.finish()

wandb:                                                                                


wandb: 
wandb: Run history:
wandb:                     elbo_train █▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                elbo_validation █▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                          epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
wandb:                kl_global_train ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:           kl_global_validation ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                 kl_local_train ▁▄▅▆▇████████▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▅▅▅
wandb:            kl_local_validation ▁▄▅▇▇▇████▇█▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆
wandb:                      kl_weight ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
wandb:      reconstruction_loss_train █▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: reconstruction_loss_validation █▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:               train_loss_epoch █▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:                train_loss_step █▅▄▄▄▃▃▂▄▂▄▃▂▃▃▃▃▁▃▃▂▂▂▃▂▁▃▃▂▃▃▂▃▃▁▂▂▂▁▂
wandb:            traine

wandb: 🚀 View run Step1_Plasma at: https://wandb.ai/inflammation/inflammation_atlas_R1/runs/nww0mr0f
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240428_190645-nww0mr0f/logs


### Extracting embedding space

In [16]:
adata.obsm['X_scVI'] = model.get_latent_representation()

## Save the results

In [17]:
if overwriteData:
    adata.write(here(f"{workDir}/{cellGroup}/results/02_{annotationLevel}_{cellGroup}_HVGsubset_scVI.h5ad"), compression="gzip")
    model.save(here(f"{workDir}/{cellGroup}/results/02_{annotationLevel}_{cellGroup}_HVGsubset_scVI_MODEL/"),
               overwrite = True, 
               save_anndata = False )   

In [18]:
# To load the model do (after loading corresponding adata):
# model = scvi.model.SCVI.load(here('{}/results/01_{}_HVGsubset_scVI_MODEL/'.format(workDir, cellGroup)), 
#                              adata, 
#                              use_gpu=True)